In [71]:
import numpy as np
from scipy.linalg import expm

# covariance example [https://www.youtube.com/watch?v=9B5vEVjH2Pk]
nvar = 3
ncases = 5
# raw scores matrix
A = np.matrix([[90, 80, 40],
               [90, 60, 80],  
               [60, 50, 70],
               [30, 40, 70],
               [30, 20, 90]])
# unity matrix
I = np.matrix([[1,1,1,1,1],
             [1,1,1,1,1],
             [1,1,1,1,1],
             [1,1,1,1,1],
             [1,1,1,1,1]])
# unity matrix * case scores = sum, *1/ncases = mean
#A_distance=A-I%*%A*1/ncases
A_distance = np.transpose(A)-np.matmul(np.transpose(A),I)*1/ncases
# covariance matrix
A_cov = np.matmul(A_distance,A)

matrix([[ 3600.,  2400., -1200.],
        [ 2400.,  2000., -1400.],
        [-1200., -1400.,  1400.]])

In [157]:
import numpy as np
from scipy.linalg import expm

# Kalman filter process
# H = transformation matrix, R = error in the observation, Y = observation matrix
# 1. Xkp = AXk-1 + BUk + Wk -> predicted state
# 2. Pkp = APk-1AT + Qk
# 3. process cov matrix
# 4. K = Pkp HT / KPkpHT + R
# 5. Yk = CYkm + Zm
# 6. Xk = Xkp + K[Y-HXk]
# 7. Pk = (I-KH)Pkp & Xk

nvar = 2

# given
Vx0 = 280
X0 = 4000
Vy0 = 120
Y0 = 3000

# initial conditions
Ax = 2 
DeltaT = 1
Vx = 280
DeltaX = 25
# process error in process covariance matrix
DeltaPx = 20
DeltaPvx = 5
# observation errors
DeltaX = 25
DeltaVx = 6

# 1. Xkp = AXk-1 + BUk + Wk -> predicted state
# A allows to update position and velocity
A = np.identity(nvar) 
A[:2, 1:] = DeltaT
X = np.matrix([[X0],
              [Vx0]])
B = np.matrix([[1/2*DeltaT*DeltaT],
               [DeltaT]])
Xkp = np.matmul(A,X) + B*Ax

# 2. Pkp.i = APk-1AT + Qk -> initial process cov matrix
Pkpi = np.matrix([[DeltaPx**2, DeltaPx*DeltaPvx],
                  [DeltaPx*DeltaPvx, DeltaPvx**2]])
# setting the cross terms to zero
Pkpi[:1, 1:] = 0
Pkpi[1:, :1] = 0

# 3. Pkp = APkp.iAT + QR >- Predicted / adjusted process cov matrix
Qr = 0    # error in the process of calculating the process cov matrix
Pkp = np.matmul(np.matmul(A,Pkpi),np.transpose(A))
# setting the cross terms to zero
Pkp[:1, 1:] = 0
Pkp[1:, :1] = 0

# 4. K = Pkp HT / KPkpHT + R -> Calculate the Kalman Gain
H = np.identity(nvar)          # matrix to allows to change the format of Pkp
R = np.matrix([[DeltaX**2, 0], # observation errors
               [0, DeltaVx**2]]) 
# Pkp%*%t(H) <- upper term
# H%*%Pkp*t(H)+R <- lower term
# dividing equals multiplication with inverse (solve function)
# K = Pkp%*%t(H)*solve(H%*%Pkp*t(H)+R)
K = np.matmul(np.matmul(Pkp, np.transpose(H)), 
          np.linalg.inv(np.matmul(np.matmul(H, Pkp),np.transpose(H))+R))

# 5. Yk = CYkm + Zm -> represents observed state
C = np.identity(nvar)
Zk = 0      # observation errors in mechanism (eg. electronic delays)
Ykm = np.matrix([[4260],
                 [282]])
Yk = np.matmul(C, Ykm)

# 6. Xk = Xkp + K[Y-HXk] -> calculate current state
Xk = Xkp + np.matmul(K, (Yk - np.matmul(H, Xkp)))

# 7. Pk = (I-KH)Pkp & Xk -> update process cov matrix
I = np.identity(nvar)
Pk = np.matmul((I - K*H), Pkp)

##############################################################

matrix([[ 4281.],
        [  282.]])

array([[0],
       [0],
       [0],
       [0]])